### Carrying on from 2021-09-09-AF-pdb-to-dssp.ipynb

> Day 2 of trying to align alphafold predictions with scope sequences

In [2]:
import json
import os
import pandas as pd

In [3]:
os.chdir('/cluster/gjb_lab/2472402')

In [4]:
merged_df = pd.read_csv('451_merged.csv')

In [5]:
ref_df = pd.read_csv('1497_url.csv').drop(columns=['url']) # to get pdbid from seqId

In [6]:
all_in_one_df = ref_df.merge(merged_df,how='right',on=['seqID','domain'])

In [7]:
all_in_one_df

,seqID,domain,pdbId,chain,entity,uniprotId,sequence,SS,AFpred,AA
0,24695,d1a12a_,1a12,a,1.0,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-------------S-------------TT-----BEEEEEEE-TTS...,MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...
1,24696,d1a1xa_,1a1x,a,1.0,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,-------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE-...,MAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...
2,24697,d1a62a1,1a62,a,1.0,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...
3,24706,d1aiea_,1aie,a,1.0,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...
4,24713,d1b79a_,1b79,a,1.0,P0ACB0,MASHMERDPQVAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVV...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,------------------TTTTTS------HHHHHHHHHHHHH-GG...,MAGNKPFNKQQAEPRERDPQVAGLKVPPHSIEAEQSVLGGLMLDNE...
...,...,...,...,...,...,...,...,...,...,...
446,26178,d4j9yb_,4j9y,b,1.0,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...,MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...
447,26183,d4kefa_,4kef,a,1.0,Q03048,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,------EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE-...,MSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...
448,26184,d4klia1,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...
449,26185,d4klia2,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...


In [8]:
# for how many of these scope domains do the sequence form a substring of AA - the uniprot sequence?

In [8]:
# df where scope sequence is a substring of uniprot sequence
substring_df = all_in_one_df.copy(deep=True)

In [9]:
n_is_substring = 0
for i,row in substring_df.iterrows():
    # if uniprot sequence does include scope domain
    if row.sequence in row.AA: 
        n_is_substring += 1
        startIdx = row.AA.find(row.sequence)
        endIdx = startIdx + len(row.sequence)
        # trim AF pred 
        substring_df.loc[i,'AFpred'] = substring_df.loc[i,'AFpred'][startIdx:endIdx]
        substring_df.loc[i,'AA'] = substring_df.loc[i,'AA'][startIdx:endIdx]
    else:
        substring_df.loc[i,'AFpred'] = '?'
        substring_df.loc[i,'AA'] = '?'

In [10]:
n_is_substring

148

In [11]:
substring_df

,seqID,domain,pdbId,chain,entity,uniprotId,sequence,SS,AFpred,AA
0,24695,d1a12a_,1a12,a,1.0,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-----S-------------TT-----BEEEEEEE-TTSTT-S-TT-...,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...
1,24696,d1a1xa_,1a1x,a,1.0,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,?,?
2,24697,d1a62a1,1a62,a,1.0,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,?,?
3,24706,d1aiea_,1aie,a,1.0,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,--------SHHHHHHHHHHHHHHHHHHHHHT,EYFTLQIRGRERFEMFRELNEALELKDAQAG
4,24713,d1b79a_,1b79,a,1.0,P0ACB0,MASHMERDPQVAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVV...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,?,?
...,...,...,...,...,...,...,...,...,...,...
446,26178,d4j9yb_,4j9y,b,1.0,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,?,?
447,26183,d4kefa_,4kef,a,1.0,Q03048,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,?,?
448,26184,d4klia1,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...
449,26185,d4klia2,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...


In [1]:
matches_df = substring_df[~substring_df['AFpred'].str.contains('\?')] # sequences with 100% match between SS and AFpred

NameError: name 'substring_df' is not defined

In [16]:
matches_df

,seqID,domain,pdbId,chain,entity,uniprotId,sequence,SS,AFpred,AA
0,24695,d1a12a_,1a12,a,1.0,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-----S-------------TT-----BEEEEEEE-TTSTT-S-TT-...,MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...
3,24706,d1aiea_,1aie,a,1.0,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,--------SHHHHHHHHHHHHHHHHHHHHHT,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...
7,24717,d1bkra_,1bkr,a,1.0,Q01082,KKSAKDALLLWCQMKTAGYPNVNIHNFTTSWRDGMAFNALIHKHRP...,-HHHHHHHHHHHHHTTT-TT---SSSSGGGTTSHHHHHHHHHH-GG...,---HHHHHHHHHHHHHTT-TT----SSSGGGTTSHHHHHHHHHH-G...,MTTTVATDYDNIEIQQQYSDVNNRWDVDDWDNENSSARLFERSRIK...
8,24718,d1bm8a_,1bm8,a,1.0,P39678,QIYSARYSGVDVYEFIHSTGSIMKRKKDDWVNATHILKAANFAKAK...,-EEEEEETTEEEEEEE-SS-EEEEETTT--EEHHHHHHHTT--HHH...,GEEEEEETTEEEEEEEETTEEEEEETTT--EEHHHHHHHTT--HHH...,MSNQIYSARYSGVDVYEFIHSTGSIMKRKKDDWVNATHILKAANFA...
9,24723,d1byia_,1byi,a,1.0,P13000,SKRYFVTGTDTEVGKTVASCALLQAAKAAGYRTAGYKPVASGSEKT...,-EEEEEEESSTTS-HHHHHHHHHHHHHHTT--EEEE-SEEES-B--...,-EEEEEEESSTTS-HHHHHHHHHHHHHHTT--EEEE-SEE-S-EEE...,MSKRYFVTGTDTEVGKTVASCALLQAAKAAGYRTAGYKPVASGSEK...
...,...,...,...,...,...,...,...,...,...,...
441,26168,d4htga2,4htg,a,1.0,Q43316,CVAVEQKTRTAIIRIGTRGSPLALAQAYETREKLKKKHPELVEDGA...,--HHHHHHHHHHHHHHHHTT--TTSSEEEEEEE-TTSEEEEEEEEE...,-----------SEEEEEESSHHHHHHHHHHHHHHHHH-GGGSSTTS...,MDIASSSLSQAHKVVLTRQPSSRVNTCSLGSVSAIGFSLPQISSPA...
443,26170,d4hwwa_,4hww,a,1.0,P05089,SRTIGIIGAPFSKGQPRGGVEEGPTVLRKAGLLEKLKEQECDVKDY...,--EEEEEEE---TTSS-GGGGGHHHHHHHTTHHHHHHTTT-EEEEE...,--EEEEEE----TTSS-TTGGGHHHHHHHTTHHHHHHHTT-EEEEE...,MSAKSRTIGIIGAPFSKGQPRGGVEEGPTVLRKAGLLEKLKEQECD...
448,26184,d4klia1,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...
449,26185,d4klia2,4kli,a,4.0,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...


In [23]:
test_row = all_in_one_df.loc[0]
test_row

seqID                                                    24695
domain                                                 d1a12a_
pdbId                                                     1a12
chain                                                        a
entity                                                       1
uniprotId                                               P18754
sequence     RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...
SS           -------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...
AFpred       -------------S-------------TT-----BEEEEEEE-TTS...
AA           MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...
Name: 0, dtype: object

In [25]:
startIdx = test_row.AA.find(test_row.sequence)
startIdx

8

In [27]:
endIdx = startIdx + len(test_row.sequence)
endIdx

421

In [30]:
test_row.AA[startIdx:endIdx]

'RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIPEDVVQAEAGGMHTVCLSKSGQVYSFGCNDEGALGRDTSVEGSEMVPGKVELQEKVVQVSAGDSHTAALTDDGRVFLWGSFRDNNGVIGLLEPMKKSMVPVQVQLDVPVVKVASGNDHLVMLTADGDLYTLGCGEQGQLGRVPELFANRGGRQGLERLLVPKCVMLKSRGSRGHVRFQDAFCGAYFTFAISHEGHVYGFGLSNYHQLGTPGTESCFIPQNLTSFKNSTKSWVGFSGGQHHTVCMDSEGKAYSLGRAEYGRLGLGEGAEEKSIPTLISRLPAVSSVACGASVGYAVTKDGRVFAWGMGTNYQLGTGQDEDAWSPVEMMGKQLENRVVLSVSSGGQHTVLLVKDKEQS'

In [31]:
test_row.AFpred[startIdx:endIdx]

'-----S-------------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--SS-EEEEEE-SSEEEEEETTS-EEEEE--TTSTT-S---STTTTSS-EE----S-EEEEEE-SSEEEEEETTS-EEEEE-EEETTEEEESSBBTB-EEEEEE---SS-EEEEEE-SSEEEEEETTS-EEEEE--TTSTTSSS-GGGTTTTGGG-THHHHS-EE---B-TTSSSB--EEEEEE-SSEEEEEETTS-EEEEE--TTSTT--SS-S-EEEEEE-GGG--SS--EEEEEE-SSEEEEEETTS-EEEEE--GGGTT-S-TT---EEEEEE-TT--SEEEEEE-SSEEEEEETTS-EEEEE--TTSTT-SSS-S-EEEEEE---TTTTTEEEEEEEE-SSEEEEEEEE----'

In [17]:
for i,row in matches_df.iterrows():
    assert len(row.SS)==len(row.AFpred)

AssertionError: 

It seems like the amino acid sequences match but the alphafold prediction pdb file does not

# Write 452 sequences to a text file with jpred

In [ ]:
# pdbe protein sequence
# dssp 8 
# dssp 3
# scope sequence
# jpred ss prediction
# AF sequence
# AF ss prediction

In [114]:
af_df = all_in_one_df.drop(columns=['pdbId','chain','entity'])
af_df.columns = ['seqID','domain','uniprotId','PDBe_sequence','DSSP8','AFpred_8','AF_sequence']
af_df

,seqID,domain,uniprotId,PDBe_sequence,DSSP8,AFpred_8,AF_sequence
0,24695,d1a12a_,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-------------S-------------TT-----BEEEEEEE-TTS...,MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...
1,24696,d1a1xa_,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,-------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE-...,MAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...
2,24697,d1a62a1,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...
3,24706,d1aiea_,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...
4,24713,d1b79a_,P0ACB0,MASHMERDPQVAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVV...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,------------------TTTTTS------HHHHHHHHHHHHH-GG...,MAGNKPFNKQQAEPRERDPQVAGLKVPPHSIEAEQSVLGGLMLDNE...
...,...,...,...,...,...,...,...
446,26178,d4j9yb_,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...,MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...
447,26183,d4kefa_,Q03048,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,------EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE-...,MSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...
448,26184,d4klia1,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...
449,26185,d4klia2,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...


In [106]:
for i,row in af_df.iterrows():
    assert len(row.AFpred_8) == len(row.AF_sequence)
    print(len(row.PDBe_sequence),len(row.DSSP8),len(row.AFpred_8),len(row.AF_sequence))

413 401 421 421
108 106 107 107
130 47 419 419
31 31 393 393
119 102 471 471
596 177 770 770
124 124 749 749
109 108 2364 2364
99 99 833 833
224 224 225 225
73 72 73 73
162 162 164 164
353 121 354 354
203 48 203 203
182 86 193 193
97 92 1248 1248
301 289 369 369
567 109 567 567
338 310 581 581
94 94 94 94
264 264 270 270
89 79 110 110
147 147 862 862
303 61 419 419
156 70 156 156
215 139 215 215
215 72 215 215
249 247 250 250
498 101 495 495
294 90 577 577
77 71 507 507
147 110 147 147
152 136 152 152
456 217 487 487
162 162 163 163
118 118 249 249
179 158 312 312
140 139 328 328
592 300 592 592
115 115 147 147
236 236 454 454
195 188 514 514
396 194 637 637
81 81 81 81
81 142 81 81
171 95 171 171
53 41 424 424
32 31 628 628
162 157 239 239
116 98 116 116
90 90 236 236
108 108 108 108
187 78 187 187
143 133 2491 2491
135 127 157 157
365 362 365 365
165 144 317 317
91 88 750 750
180 165 191 191
759 759 760 760
314 111 595 595
516 188 550 550
105 76 453 453
115 107 123 123
121 113 664 66

In [ ]:
jpred_table = '/homes/www-jpred/live4/public_html/results/JPred4_JNet231_RETR2_DO_NOT_DELETE/details_v4_1498.txt'

In [14]:
with open(jpred_table, 'r') as f:
    rows = f.read().splitlines()

In [66]:
jpred_df_list = []

for row in rows:
    row_array = row.split(' ')
    domain = row_array[0]
    sspred = row_array[2]
    sequen = row_array[4]
    jpred_df_list.append(pd.Series({'domain':domain,'jpred_sequence':sequen,'jpred_3':sspred}))

In [67]:
jpred_df = pd.concat(jpred_df_list,axis=1).T
jpred_df

,domain,jpred_sequence,jpred_3
0,d1a12a_,KKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIP...,-----EEEEEEE----EEEEE-------------------EEE---...
1,d1a1xa_,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,----------EEEEE---EEEE----EEEEEEEE----EEEEEEE-...
2,d1a62a1,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSGE,--HHH-----HHHHHHHHHH----------HHHHHHHHHHHHHH---
3,d1a9xa1,EVGATGFDPKVSLDDPEALTKIRRELKDAGADRIWYIADAFRAGLS...,------------------HHHHHHHHH----HHHHHHHHHHH----...
4,d1a9xb1,IKSALLVLEDGTQFHGRAIGATGSAVGEVVFNTSMTGYQEILTDPS...,---EEEEE----EEEEEEE-----EEEEEEEE------EEEE----...
...,...,...,...
1493,e1n13.1B,IMPPEAEIVPLPKLPMGALVPTAYGYIISDVPGETISAAISVAIPK...,------EEE------------HHEEEEEE-----EEEEEEEEEEE-...
1494,e1qd6.1C,FTLYPYDTNYLIYTQTSDLNKEAIASYDWAENARKDEVKFQLSLAF...,---------EEEEEEE---------------------EEEEEEEEE...
1495,e1qqp.12,DKKTTTLLEDRILTTRNGHTTSTTQSSVGVTYGYATAEDFVSGPNT...,-----------EEEE-----EE--------EEE-------------...
1496,e1qtn.1A,DKVYQMKSKPRGYCLIINNHNFAKAREKVPKLHSIRDRNGTHLDAG...,---E-------EEEEEEE-----------------------HHHHH...


In [68]:
for i,row in jpred_df.iterrows():
    assert len(row.jpred_3) == len(row.jpred_sequence)

In [115]:
sum_df = af_df.merge(jpred_df, how='left', on='domain')
sum_df

,seqID,domain,uniprotId,PDBe_sequence,DSSP8,AFpred_8,AF_sequence,jpred_sequence,jpred_3
0,24695,d1a12a_,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-------------S-------------TT-----BEEEEEEE-TTS...,MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...,KKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIP...,-----EEEEEEE----EEEEE-------------------EEE---...
1,24696,d1a1xa_,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,-------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE-...,MAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,----------EEEEE---EEEE----EEEEEEEE----EEEEEEE-...
2,24697,d1a62a1,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSGE,--HHH-----HHHHHHHHHH----------HHHHHHHHHHHHHH---
3,24706,d1aiea_,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,EYFTLQIRGRERFEMFRELNEALELKDAQAG,-EEEEEEE--HHHHHHHHHHHHHHHHH----
4,24713,d1b79a_,P0ACB0,MASHMERDPQVAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVV...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,------------------TTTTTS------HHHHHHHHHHHHH-GG...,MAGNKPFNKQQAEPRERDPQVAGLKVPPHSIEAEQSVLGGLMLDNE...,PPHSIEAEQSVLGGLMLDNERWDDVAERVVADDFYTRPHRHIFTEM...,----HHHHHHHHHHHH----HHHHHHHHH----HHHHHHHHHHHHH...
...,...,...,...,...,...,...,...,...,...
446,26178,d4j9yb_,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...,MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...,GRKLELTKAEDTQLTKRVKNAAANVLRETWLIYKNTKLVKKIDHAK...,---------HHHHH-HHHHHHHHHHHHHHHHHHHHHH------HHH...
447,26183,d4kefa_,Q03048,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,------EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE-...,MSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,---------HHHHHHHHHHH-----EEEEEEEE---EEEEEE----...
448,26184,d4klia1,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,TLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVIAKYPHKIKS...,---HHHHHHHHHHHHHHHHH-----HHHHHHHHHHHHH--------...
449,26185,d4klia2,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,DTSSSINFLTRVSGIGPSAARKFVDEGIKTLEDLRKNEDKLNHHQR...,-HHHHHHHHH------HHHHHHHHHH----HHHHH-------HHHH...


### Now obtain 3 state dssp

In [5]:
import re
def reduce_dssp(sec_string):
    sec_string = re.compile(r'[ST_bGIC?]').sub('-',sec_string) # convert all known non-sheet, non-helix structures to coil
    sec_string = re.compile(r'[B]').sub('E',sec_string) # convert B forms to E
    sec_string = re.compile(r'~[HE]').sub('-',sec_string) # convert everything not H or E to coil for backup
    return sec_string

In [110]:
d=10
# test the reduce_dssp function 
test_dssp8 = sum_df.DSSP8[d]
print(test_dssp8)
print(reduce_dssp(test_dssp8))

---EEEEEEE---SHHHHHHHHHHHHTTTTSEEEEEEETTTTEEEEEESS-HHHHHHHHHTTSS-EEEEEE-
---EEEEEEE----HHHHHHHHHHHH-----EEEEEEE----EEEEEE---HHHHHHHHH-----EEEEEE-


### Run below cell to write out summary table

In [116]:
with open('summary_table.txt','w') as f:
    for i,row in sum_df.iterrows():
        print('seqID          :',row.seqID,file=f)
        print('domain         :',row.domain,file=f)
        print('UniprotId      :',row.uniprotId,file=f)
        print('PDBe_sequence  :',row.PDBe_sequence,file=f)
        print('jpred_sequence :',row.jpred_sequence,file=f)
        print('DSSP_8         :',row.DSSP8,file=f)
        print('DSSP_3         :',reduce_dssp(row.DSSP8),file=f)
        print('jpred_3        :',row.jpred_3,file=f)
        print('AF_sequence    :',row.AF_sequence,file=f)
        print('AFpred_8       :',row.AFpred_8,file=f)
        print('AFpred_3       :',reduce_dssp(row.AFpred_8),file=f)

In [118]:
sum_df.to_csv('summary_table.csv',index=False)

# Align alphafold predictions to jpred predictions

In [6]:
import pylcs
import os
import pandas as pd
import re


In [7]:
def reduce_dssp(sec_string):
    sec_string = re.compile(r'[ST_bGIC?]').sub('-',sec_string) # convert all known non-sheet, non-helix structures to coil
    sec_string = re.compile(r'[B]').sub('E',sec_string) # convert B forms to E
    sec_string = re.compile(r'~[HE]').sub('-',sec_string) # convert everything not H or E to coil for backup
    return sec_string

In [2]:
# lcs_align - alignment based on longest common subsequence
# assumption - no indels i.e. no gaps
# returns start and end indices to slice compstring with
# distance between start and end indices is exactly the length of refstring
# in practice 
# ... refstring is the jpred protein sequence,
# ... and compstring is the uniprot/alphafold sequence
# this function is used to trim uniprot sequence
# if threshold e.g. 0.9 is not met, then start and end indices of 0 and -1 is returned
def lcs_align(refstring, compstring, threshold = 0):
    assert threshold <= 1
    lref = len(refstring) 
    lcomp = len(compstring)
    assert lref <= lcomp
    lsearch = lcomp - lref + 1 # how many amino acid is stringc longer than stringb by
    start,end = 0, lref # index to slice comparator string 
    best_overlap = pylcs.lcs(refstring,compstring[start:end]) # alignment start from N terminus
    best_offset = 0
    for offset in range(0,lsearch):
        # compare percentage similarity
        assert end+offset <= lcomp
        test_window = compstring[start+offset:end+offset]
        test_overlap = pylcs.lcs(refstring, test_window)
        if test_overlap > best_overlap:
            best_overlap = test_overlap
            best_offset = offset
        else:
            pass
    
    # sanity check on best_offset by using it to slice compstring
    best_window = compstring[best_offset:best_offset + lref]
    # check we indeed have the best answer (probably unecessary but I'm not a computer scientist)
    assert pylcs.lcs(best_window, refstring) == best_overlap
    
    if best_overlap >= threshold * lref:
        return best_offset
    else:
        # threshold is not met
        return -1

# tests
# test 1 - compare nothing
assert lcs_align('','',1)==0
# test 2 - compare simple strings
assert lcs_align('ABC','DABAABCAB',1)==4
# test 3 - fail threshold
assert lcs_align('ABDDC','DABAABCAB',0.8)==-1
# test 4 - compare normal length strings
assert lcs_align('NKKPVNSWTCEDFLAVDESFQPTAVGFAEALNNKDKPEDAVLDVQGIATVTPAIVQACTQDKQANFKDKVKGEWDKIKK','MKKVLGVILGGLLLLPVVSNAADAQKAADNKKPVNSWTCEDFLAVDESFQPTAVGFAEALNNKDKPEDAVLDVQGIATVTPAIVQACTQDKQANFKDKVKGEWDKIKKDM',1)==29
# test 5 
assert lcs_align('GSASPTPPYLKWAESLHSLLDDQDGISLFRTFLKQEGCADLLDFWFACTGFRKLEPCDSNEEKRLKLARAIYRKYILDNNGIVSRQTKPATKSFIKGCIMKQLIDPAMFDQAQTEIQATMEENTYPSFLKSDIYLEYTRTGSESPKV',
                 'MNIQEQGFPLDLGASFTEDAPRPPVPGEEGELVSTDPRPASYSFCSGKGVGIKGETSTATPRRSDLDLGYEPEGSASPTPPYLKWAESLHSLLDDQDGISLFRTFLKQEGCADLLDFWFACTGFRKLEPCDSNEEKRLKLARAIYRKYILDNNGIVSRQTKPATKSFIKGCIMKQLIDPAMFDQAQTEIQATMEENTYPSFLKSDIYLEYTRTGSESPKVCSDQSSGSGTGKGISGYLPTLNEDEEWKCDQDMDEDDGRDAAPPGRLPQKLLLETAAPRVSSSRRYSEGREFRYGSWREPVNPYYVNAGYALAPATSANDSEQQSLSSDADTLSLTDSSVDGIPPYRIRKQHRREMQESVQVNGRVPLPHIPRTYRVPKEVRVEPQKFAEELIHRLEAVQRTREAEEKLEERLKRVRMEEEGEDGDPSSGPPGPCHKLPPAPAWHHFPPRCVDMGCAGLRDAHEENPESILDEHVQRVLRTPGRQSPGPGHRSPDSGHVAKMPVALGGAASGHGKHVPKSGAKLDAAGLHHHRHVHHHVHHSTARPKEQVEAEATRRAQSSFAWGLEPHSHGARSRGYSESVGAAPNASDGLAHSGKVGVACKRNAKKAESGKSASTEVPGASEDAEKNQKIMQWIIEGEKEISRHRRTGHGSSGTRKPQPHENSRPLSLEHPWAGPQLRTSVQPSHLFIQDPTMPPHPAPNPLTQLEEARRRLEEEEKRASRAPSKQRYVQEVMRRGRACVRPACAPVLHVVPAVSDMELSETETRSQRKVGGGSAQPCDSIVVAYYFCGEPIPYRTLVRGRAVTLGQFKELLTKKGSYRYYFKKVSDEFDCGVVFEEVREDEAVLPVFEEKIIGKVEKVD',
                 1)==73
# test 6 
# fixed: lsearch = lcomp - lref +1 
assert lcs_align('RIPTDPTMYRFYEMLQVYGTTLKALVHEKFGDGIISAINFKLDVKKVADPEGGERAVITLDGKYLPTKPF',
                 'MIQSQINRNIRLDLADAILLSKAKKDLSFAEIADGTGLAEAFVTAALLGQQALPADAARLVGAKLDLDEDSILLLQMIPLRGCIDDRIPTDPTMYRFYEMLQVYGTTLKALVHEKFGDGIISAINFKLDVKKVADPEGGERAVITLDGKYLPTKPF',1
                )==86
# test 7
assert lcs_align('NERNISRLWRAFRTVKEMVKDRGYFITQEEVELPLEDFKAKYCDSMGRPQRKMMSFQANPTEESISKFPDMGSLWVEFCDEPSVGVKTMKTFVIHIQEKNFQTGIFVYQNNITPSAMKLVPSIPPATIETFNEAALVVN',
                 'MDQENERNISRLWRAFRTVKEMVKDRGYFITQEEVELPLEDFKAKYCDSMGRPQRKMMSFQANPTEESISKFPDMGSLWVEFCDEPSVGVKTMKTFVIHIQEKNFQTGIFVYQNNITPSAMKLVPSIPPATIETFNEAALVVNITHHELVPKHIRLSSDEKRELLKRYRLKESQLPRIQRADPVALYLGLKRGEVVKIIRKSETSGRYASYRICM',
                 1
                )==4

Run lcs_align function on jpred

In [3]:
os.chdir('/cluster/gjb_lab/2472402')
sum_df = pd.read_csv('summary_table.csv')
sum_df

,seqID,domain,uniprotId,PDBe_sequence,DSSP8,AFpred_8,AF_sequence,jpred_sequence,jpred_3
0,24695,d1a12a_,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-------------S-------------TT-----BEEEEEEE-TTS...,MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...,KKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIP...,-----EEEEEEE----EEEEE-------------------EEE---...
1,24696,d1a1xa_,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,-------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE-...,MAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,----------EEEEE---EEEE----EEEEEEEE----EEEEEEE-...
2,24697,d1a62a1,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSGE,--HHH-----HHHHHHHHHH----------HHHHHHHHHHHHHH---
3,24706,d1aiea_,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,EYFTLQIRGRERFEMFRELNEALELKDAQAG,-EEEEEEE--HHHHHHHHHHHHHHHHH----
4,24713,d1b79a_,P0ACB0,MASHMERDPQVAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVV...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,------------------TTTTTS------HHHHHHHHHHHHH-GG...,MAGNKPFNKQQAEPRERDPQVAGLKVPPHSIEAEQSVLGGLMLDNE...,PPHSIEAEQSVLGGLMLDNERWDDVAERVVADDFYTRPHRHIFTEM...,----HHHHHHHHHHHH----HHHHHHHHH----HHHHHHHHHHHHH...
...,...,...,...,...,...,...,...,...,...
446,26178,d4j9yb_,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...,MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...,GRKLELTKAEDTQLTKRVKNAAANVLRETWLIYKNTKLVKKIDHAK...,---------HHHHH-HHHHHHHHHHHHHHHHHHHHHH------HHH...
447,26183,d4kefa_,Q03048,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,------EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE-...,MSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,---------HHHHHHHHHHH-----EEEEEEEE---EEEEEE----...
448,26184,d4klia1,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,TLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVIAKYPHKIKS...,---HHHHHHHHHHHHHHHHH-----HHHHHHHHHHHHH--------...
449,26185,d4klia2,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,HHHHHHHHHTTSTT--HHHHHHHHHTT--SHHHHHHTGGGS-HHHH...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,DTSSSINFLTRVSGIGPSAARKFVDEGIKTLEDLRKNEDKLNHHQR...,-HHHHHHHHH------HHHHHHHHHH----HHHHH-------HHHH...


In [4]:
sum_df_trunc = sum_df.copy(deep=True) # truncated version

In [17]:
for i,row in sum_df_trunc.iterrows():
    sum_df_trunc.loc[i,'DSSP3'] = reduce_dssp(sum_df_trunc.loc[i,'DSSP8'])
    if len(row.jpred_sequence) <= len(row.AF_sequence):
        threshold=0.9
        start = lcs_align(row.jpred_sequence, row.AF_sequence, threshold)
        if start >= 0:
            end = start + len(row.jpred_sequence)
            sum_df_trunc.loc[i,'AF_sequence_trunc'] = sum_df_trunc.loc[i,'AF_sequence'][start:end]
            sum_df_trunc.loc[i,'AFpred_8_trunc'] = sum_df_trunc.loc[i,'AFpred_8'][start:end]
            sum_df_trunc.loc[i,'AFpred_3_trunc'] = reduce_dssp(sum_df_trunc.loc[i,'AFpred_8_trunc'])
        else:
            sum_df_trunc.loc[i,'AF_sequence_trunc'] = 'no subsequence with >%.2f similarity found' % threshold
            sum_df_trunc.loc[i,'AFpred_8_trunc'] = 'no subsequence with >%.2f similarity found' % threshold
            sum_df_trunc.loc[i,'AFpred_3_trunc'] = 'no subsequence with >%.2f similarity found' % threshold
    else:
        sum_df_trunc.loc[i,'AF_sequence_trunc'] = '? length of alphafold protein sequence is shorter than jpred protein sequence. %d vs %d' % (len(row.AF_sequence), len(row.jpred_sequence))
        sum_df_trunc.loc[i,'AFpred_8_trunc'] = '? length of alphafold protein sequence is shorter than jpred protein sequence'
        sum_df_trunc.loc[i,'AFpred_3_trunc'] = '? length of alphafold protein sequence is shorter than jpred protein sequence'

In [18]:
sum_df_trunc

,seqID,domain,uniprotId,PDBe_sequence,DSSP8,AFpred_8,AF_sequence,jpred_sequence,jpred_3,DSSP3,AF_sequence_trunc,AFpred_8_trunc,AFpred_3_trunc
0,24695,d1a12a_,P18754,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,-------------S-------------TT-----BEEEEEEE-TTS...,MSPKRIAKRRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVG...,KKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIP...,-----EEEEEEE----EEEEE-------------------EEE---...,--------------EEEEEEEE-------------EEEEEEEE---...,KKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENVMERKKPALVSIP...,-------TT-----BEEEEEEE-TTSTT-S-TT--EEEEEEEE--S...,--------------EEEEEEEE-------------EEEEEEEE---...
1,24696,d1a1xa_,P56278,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,---------SEEEEEETTEEEETTS-EEEEEEEE-SS-EEEEEE--...,-------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE-...,MAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQ...,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,----------EEEEE---EEEE----EEEEEEEE----EEEEEEE-...,----------EEEEEE--EEEE----EEEEEEEE----EEEEEE--...,AGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQQI...,------S--SEEEEEETTEEEETT--EEEEEEEEETTEEEEEEE--...,----------EEEEEE--EEEE----EEEEEEEEE--EEEEEEE--...
2,24697,d1a62a1,P0AG30,*NLTELKNTPVSELITLGEN*GLENLAR*RKQDIIFAILKQHAKSG...,-BHHHHHTS-HHHHHHHHHTTT----TTS-HHHHHHHHHHHHHHTT-,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSGE,--HHH-----HHHHHHHHHH----------HHHHHHHHHHHHHH---,-EHHHHH---HHHHHHHHH-----------HHHHHHHHHHHHHH---,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSGE,-BHHHHHHS-HHHHHHHHHHTT--SGGGS-HHHHHHHHHHHHHHTT-,-EHHHHHH--HHHHHHHHHH----------HHHHHHHHHHHHHH---
3,24706,d1aiea_,P04637,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHH--,---------------TTTHHHHHHTS-S--S---------------...,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,EYFTLQIRGRERFEMFRELNEALELKDAQAG,-EEEEEEE--HHHHHHHHHHHHHHHHH----,---------HHHHHHHHHHHHHHHHHHHH--,EYFTLQIRGRERFEMFRELNEALELKDAQAG,--------SHHHHHHHHHHHHHHHHHHHHHT,---------HHHHHHHHHHHHHHHHHHHHH-
4,24713,d1b79a_,P0ACB0,MASHMERDPQVAGLKVPPHSIEAEQSVLGGLMLDNERWDDVAERVV...,---HHHHHHHHHHHHHH-GGGHHHHHTT--GGGSSSHHHHHHHHHH...,------------------TTTTTS------HHHHHHHHHHHHH-GG...,MAGNKPFNKQQAEPRERDPQVAGLKVPPHSIEAEQSVLGGLMLDNE...,PPHSIEAEQSVLGGLMLDNERWDDVAERVVADDFYTRPHRHIFTEM...,----HHHHHHHHHHHH----HHHHHHHHH----HHHHHHHHHHHHH...,---HHHHHHHHHHHHHH----HHHHH----------HHHHHHHHHH...,PPHSIEAEQSVLGGLMLDNERWDDVAERVVADDFYTRPHRHIFTEM...,----HHHHHHHHHHHHH-GGGHHHHHTT--GGG-SSHHHHHHHHHH...,----HHHHHHHHHHHHH----HHHHH----------HHHHHHHHHH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,26178,d4j9yb_,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...,MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...,GRKLELTKAEDTQLTKRVKNAAANVLRETWLIYKNTKLVKKIDHAK...,---------HHHHH-HHHHHHHHHHHHHHHHHHHHHH------HHH...,-----EE----HHHHHHHHHHHHHHHHHHHHHHHHHH------HHH...,RKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTKLV...,HHHS--HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHS-...,HHH---HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH--...
447,26183,d4kefa_,Q03048,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,--SS--EE-HHHHHHHHHHHHH---SEEEEEE-TTSSEEEEEEEE-...,------EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE-...,MSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,GSRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETST...,---------HHHHHHHHHHH-----EEEEEEEE---EEEEEE----...,------EE-HHHHHHHHHHHHH----EEEEEE-----EEEEEEEE-...,SRSGVAVADESLTAFNDLKLGKKYKFILFGLNDAKTEIVVKETSTD...,-----EE-HHHHHHHHHHHTT---SEEEEEE-TTSSEEEEEEEE--...,-----EE-HHHHHHHHHHH------EEEEEE-----EEEEEEEE--...
448,26184,d4klia1,P06746,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,-TTHHHHHHHHHHHHHHHHTT--HHHHHHHHHHHHHHHH--S----...,----------TTHHHHHHHHHHHHHHHHTS--HHHHHHHHHHHHHH...,MSKRKAPQETLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVI...,TLNGGITDMLTELANFEKNVSQAIHKYNAYRKAASVIAKYPHKIKS...,---HHHHHHHHHHHHHHHHH

In [19]:
sum_df_trunc[sum_df_trunc.AF_sequence_trunc.str.contains('\?')]

,seqID,domain,uniprotId,PDBe_sequence,DSSP8,AFpred_8,AF_sequence,jpred_sequence,jpred_3,DSSP3,AF_sequence_trunc,AFpred_8_trunc,AFpred_3_trunc
44,24798,d1fm0e_,P30748,MIKVLFFAQVRELVGTDATEVAADFPTVEALRQHMAAQSDRWALAL...,--EEEEEESS---HHHHHHHHT--TT--EEEEEEEE-----EEEEE...,-EEEEE-HHHHHHHT-SEEEE-S--SBHHHHHHHHHTT-HHHHHHT...,MIKVLFFAQVRELVGTDATEVAADFPTVEALRQHMAAQSDRWALAL...,AETKIVVGPQPFSVGEEYPWLAERDEDGAVVTFTGKVRVNALTLEH...,---EEEEE-----HHHHHHHHH------EEEEEEEEEEEEEEEE--...,--EEEEEE-----HHHHHHHH-------EEEEEEEE-----EEEEE...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...
104,24970,d1lm8v_,Q15370,MDVFLMIRRHKTTIFTDAKESSTVFELKRIVEGILKRPPDEQRLYK...,--SS-------EEEEEEEE-SSS-EEEEEE-TTS-EEE---B-TTE...,-EEEEEEEEBTEEEEEEEETTSBHHHHHHHHHHHH---GGGEEEEE...,MDVFLMIRRHKTTIFTDAKESSTVFELKRIVEGILKRPPDEQRLYK...,RPVLRSVNSREPSQVIFCNRSPRVVLPVWLNFDGEPQPYPTLPPGT...,----------EEEEEEEEE----EEEEEEE-----EEE--------...,-----------EEEEEEEE-----EEEEEE-----EEE---E---E...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...
157,25152,d1ri6a_,P52697,MSLKQTVYIASPESQQIHVWNLNHEGALTLTQVVDVPGQVQPMVVS...,--EEEEEEEEGGGTEEEEEEE-TTS-EEEEEEEE-SS----EEE-T...,--EEEEEEEGGGTEEEEEEE-TTS-EEEEEEEE-SSEEEEEEE-TT...,MKQTVYIASPESQQIHVWNLNHEGALTLTQVVDVPGQVQPMVVSPD...,SLKQTVYIASPESQQIHVWNLNHEGALTLTQVVDVPGQVQPMVVSP...,----EEEEEE-----EEEEEE-----EEEEEEE------EEEEEE-...,--EEEEEEEE----EEEEEEE-----EEEEEEEE-------EEE--...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...
228,25389,d1yfqa_,P26449,MQIVQIEQAPKDYISDIKIIPSKSLLLITSWDGSLTVYKFDIQAKN...,-EEEE-SS--SS-EEEEEEEGGGTEEEEEETTSEEEEEEEETTTTE...,-EEEE-TT--SS-EEEEEEEGGGTEEEEEETTSEEEEEEEETTTTE...,MQIVQIEQAPKDYISDIKIIPSKSLLLITSWDGSLTVYKFDIQAKN...,MQIVQIEQAPKDYISDIKIIPSKSLLLITSWDGSLTVYKFDIQAKN...,---EEE-------EEEEEE-----EEEEE----EEEEEEE------...,-EEEE--------EEEEEEE----EEEEEE---EEEEEEEE----E...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...
235,25418,d1zhxa_,P35844,GAMDPSQYASSSSWTSFLKSIASFNGDLSSLSAPPFILSPISLTEF...,--HHHHTTSHHHHHHHHTTTT--S-GGGS---GGGEEEEEGGGGGG...,--TTTT-HHHHHHHHHHHT--S-GGGS---GGGEEEEEGGGGGGGG...,MSQYASSSSWTSFLKSIASFNGDLSSLSAPPFILSPISLTEFSQYW...,MDPSQYASSSSWTSFLKSIASFNGDLSSLSAPPFILSPISLTEFSQ...,----------HHHHHHHHH---------EE--HHHH-HHHHHHHHH...,--HHHH---HHHHHHHH------------------EEEEE------...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...,? length of alphafold protein sequence is shor...


Issue is that there are still many sequences not aligned

## Print to summary table, this timed called summary_table_trunc.txt

In [20]:
with open('summary_table_trunc.txt','w') as f:
    print('NOTE: AlphaFold predictions have been truncated to align as closely to jpred_sequences',file=f)
    print('NOTE: Accuracy of -1.0 means that the lengths of sequences differ',file=f)
    for i,row in sum_df_trunc.iterrows():
        print('seqID          :',row.seqID,file=f)
        print('domain         :',row.domain,file=f)
        print('UniprotId      :',row.uniprotId,file=f)
        print('PDBe_sequence  :',row.PDBe_sequence,file=f)
        print('jpred_sequence :',row.jpred_sequence,file=f)
        print('DSSP_8         :',row.DSSP8,file=f)
        print('DSSP_3         :',row.DSSP3,file=f)
        print('jpred_3        :',row.jpred_3,file=f)
        print('AF_sequence    :',row.AF_sequence_trunc,file=f)
        print('AFpred_8       :',row.AFpred_8_trunc,file=f)
        print('AFpred_3       :',row.AFpred_3_trunc,file=f)
        #print('Q3 acc Jpred   :',row.jpred_Q3_acc,file=f)
        #print('Q3 acc AF      :',row.AF_Q3_acc,file=f)

Save as csv

In [21]:
sum_df_trunc.to_csv('summary_table_trunc.csv',index=False)

# End of work on 09/09/2021

## Add accuracy scores to table (FIXME)

In [11]:
def score(ref,comp):
    if len(ref)==len(comp):
        n_match=0
        if len(ref)==0:
            pass
        else:
            for i in range(0,len(ref)):
                ref_letter = ref[i:i+1]
                comp_letter = comp[i:i+1]
                if ref_letter == comp_letter:
                    n_match += 1
                else:
                    pass
        return n_match/len(ref)
    else:
        return '-1'

In [12]:
for i,row in sum_df_trunc.iterrows():
        sum_df_trunc.loc[i,'jpred_Q3_acc'] = score(row.DSSP3, row.jpred_3)
        sum_df_trunc.loc[i,'AF_Q3_acc'] = score(row.DSSP3, row.AFpred_3_trunc)

Calculate average Q3 accuracy

In [16]:
accuracy_df = sum_df_trunc[~sum_df_trunc.AF_sequence_trunc.str.contains('\?')]

In [24]:
AF_acc_series = accuracy_df.AF_Q3_acc.astype(float)
jpred_acc_series = accuracy_df.jpred_Q3_acc.astype(float)

In [25]:
AF_mean = AF_acc_series.mean()
AF_std = AF_acc_series.std()
jpred_mean = jpred_acc_series.mean()
jpred_std = jpred_acc_series.std()
print(f'Jpred mean ± standard deviation: {jpred_mean} ± {jpred_std}')
print(f'AF    mean ± standard deviation: {AF_mean} ± {AF_std}')

Jpred mean ± standard deviation: 0.8354474674932473 ± 0.06923381399459008
AF    mean ± standard deviation: 0.6763363382014447 ± 0.621913915178828


Turns out many alignments are wrong

For example this one

In [36]:
problem_row = sum_df_trunc.loc[sum_df_trunc['seqID']==26178]
problem_row

,seqID,domain,uniprotId,PDBe_sequence,DSSP8,AFpred_8,AF_sequence,jpred_sequence,jpred_3,DSSP3,AF_sequence_trunc,AFpred_8_trunc,AFpred_3_trunc,jpred_Q3_acc,AF_Q3_acc
446,26178,d4j9yb_,P70604,MGRKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTK...,--TT-EE----HHHHHHHHHHHHHHHHHHHHHHHHHHS-SS--HHH...,-TT-------------------------------------------...,MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...,GRKLELTKAEDTQLTKRVKNAAANVLRETWLIYKNTKLVKKIDHAK...,---------HHHHH-HHHHHHHHHHHHHHHHHHHHHH------HHH...,-----EE----HHHHHHHHHHHHHHHHHHHHHHHHHH------HHH...,RKLELTKAEKHVHNFMMDTQLTKRVKNAAANVLRETWLIYKNTKLV...,HHHS--HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHS-...,HHH---HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH--...,0.94382,0.696629


In [39]:
problem_row.AF_sequence

'446    MSSCRYNGGVMRPLSNLSSSRRNLHEMDSEAQPLQPPASVVGGGGG...\nName: AF_sequence, dtype: object'

In [40]:
problem_row.jpred_sequence

446    GRKLELTKAEDTQLTKRVKNAAANVLRETWLIYKNTKLVKKIDHAK...
Name: jpred_sequence, dtype: object

In [43]:
sum_df_trunc.loc[sum_df_trunc['seqID']==24741]

,seqID,domain,uniprotId,PDBe_sequence,DSSP8,AFpred_8,AF_sequence,jpred_sequence,jpred_3,DSSP3,AF_sequence_trunc,AFpred_8_trunc,AFpred_3_trunc,jpred_Q3_acc,AF_Q3_acc
14,24741,d1cxzb_,P61586,SMAAIRKKLVIVGDVACGKTCLLIVFSKDQFPEVYVPTVFENYVAD...,--HHHHTT-TTS-TTSHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH...,---EEEEEEEEE-TTSSHHHHHHHHHHS---SS----SEEEEEEEE...,MAAIRKKLVIVGDGACGKTCLLIVFSKDQFPEVYVPTVFENYVADI...,WSLLEQLGLAGADLAAPGVQQQLELERERLRREIRKELKLKEGAEN...,--HHHHH--------------HHHHHHHHHHHHHHHHHHHHHHHHH...,--HHHH----------HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH...,,,,0.825581,-1
